# **Coding the model environment for the relevant problem and displaying it on the screen**

> **NumPy** kütüphanesi, özellikle sayısal işlemler ve dizilerle çalışmak için kullanılır.

> Ortamın **11x11** boyutlarında olduğunu belirtiyoruz.

> Her bir durum (11x11 griddeki her hücre) ve her bir eylem (yukarı, sağ, aşağı, sol) için Q değerleri **sıfır**dan başlatılır. q_degerleri matrisi, **11x11x4** boyutlarındadır.

> Q-öğrenme algoritmasının kullanabileceği **hareketler (eylemler)** belirlenir.

> 11x11 boyutlarında, başlangıçta tüm hücreler için ödül değerlerini **-100** olarak ayarlayan bir **ödül matrisi** oluşturulur.

> Hedef durum olan **(0, 5)** konumunun ödülü **100** olarak ayarlanır.

> **Geçitler**, ajan için **geçilebilir hücreleri** belirler. Her bir satır indeksi için geçilebilir sütun indekslerinin bir listesi oluşturulur.

> Geçitlerdeki hücrelerin ödülleri **-1** olarak ayarlanır, böylece ajan bu hücrelerden geçerken sadece *küçük bir ceza* alır.

> Ödül matrisi **satır satır** ekrana yazdırılır.

In [1]:
#Gerekli kütüphanenin yüklenmesi
import numpy as np
#Ortamın boyutlarının belirlenmesi
ortam_satir_sayisi = 11
ortam_sutun_sayisi = 11
#Q-değerlerinin başlangıçta sıfır olarak ayarlanması
q_degerleri = np.zeros((ortam_satir_sayisi, ortam_sutun_sayisi, 4))
#Hareketlerin tanımlanması
hareketler = ['yukari', 'sag', 'asagi', 'sol']
#Ödül matrisinin oluşturulması
oduller = np.full((ortam_satir_sayisi, ortam_sutun_sayisi), -100.)
#Hedef durumun ödülünün ayarlanması
oduller[0, 5] = 100.
#Geçitlerin tanımlanması
gecitler = {} 
gecitler[1] = [i for i in range(1, 10)]
gecitler[2] = [1, 7, 9]
gecitler[3] = [i for i in range(1, 8)]
gecitler[3].append(9)
gecitler[4] = [3, 7]
gecitler[5] = [i for i in range(11)]
gecitler[6] = [5]
gecitler[7] = [i for i in range(1, 10)]
gecitler[8] = [3, 7]
gecitler[9] = [i for i in range(11)]

#Geçitlerin ödüllerinin ayarlanması
for satir_indeks in range(1, 10):
  for sutun_indeks in gecitler[satir_indeks]:
    oduller[satir_indeks, sutun_indeks] = -1.
    
#Ödül matrisinin ekrana yazdırılması
for satir in oduller:
  print(satir)

[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]


# **Coding of robot movement mechanisms**

> Ardından robotun her hareketinde beyaz karede kalıp kalmadığını belirleyen, her yeni hareket döngülerinde başlangıç noktası atamasını yapan ve her yeni karede bir sonraki hareket noktasını yeni Q tablosu ile belirleyen fonksiyonlar tanımlanır.

> **engel_mi** fonksiyonu robotun hareket etmesi sonrası gelinen hücrenin engel (yani kargo indirilen siyah ya da hedef yeşil kare) olup olmadığını tespit etmektedir.
> 
> *gecerli_satir_indeks ve gecerli_sutun_indeks* parametreleri, kontrol edilmesi gereken hücrenin indeksleridir.Eğer ödül matrisi (oduller) bu hücrede **-1** değerine sahipse, bu hücrenin geçit (engel değil) olduğu anlamına gelir ve fonksiyon **False** döner. Aksi takdirde, bu hücrenin bir engel olduğu anlamına gelir ve fonksiyon True döner.

> **baslangic_belirle** fonksiyonu robotun her yeni öğrenme döngüsünde rastgele beyaz karelerden birinden başlamasını sağlanmaktadır. Bunun için daha önce yazılan *engel_mi* fonksiyonundan da faydalanılmaktadır.
> 
> Önce, **rastgele** bir satır ve sütun indeksi seçilir.Seçilen hücre engel mi (engel_mi fonksiyonu kullanılarak) kontrol edilir.Eğer seçilen hücre bir engel ise, **yeni** bir satır ve sütun indeksi seçilir. Bu işlem, engel olmayan bir hücre bulunana kadar devam eder.Fonksiyon, engel olmayan bir hücrenin satır ve sütun indekslerini döner.

> **sonraki_hareket_belirle** fonksiyonu robotun mevcut konumundan sonra hareket edebileceği sonraki konumu eğitim sonunda elde edilen Q tablosundan belirlemektedir. 
> Bu noktada buradaki kod yapısına özel olarak belirlenen **epsilon** değeri de *şans faktörü* olarak dikkate alınmaktadır. Buna göre, epsilon değerinden küçük olan rastgele bir değer üretilirse puan tablosundan **maksimum değerin**, eşit ya da büyük rastgele bir değer üretilirse de bir başka **rastgele değerin (puan tablosu dikkate alınmadan)** döndürülmesi sağlanmaktadır. 
> 
> gecerli_satir_indeks ve gecerli_sutun_indeks parametreleri, mevcut durumun indeksleridir. **epsilon** parametresi, keşif oranını belirler *(0 ile 1 arasında bir değerdir)*. Eğer rastgele seçilen bir sayı **(0 ile 1 arasında)** epsilon'dan küçükse, en yüksek Q-değerine sahip hareket seçilir **(np.argmax kullanılarak)**. Aksi takdirde, rastgele bir hareket seçilir **(np.random.randint(4) kullanılarak)**. Burada 4, dört olası hareketin **(yukarı, sağ, aşağı, sol)** indekslerini temsil eder.

In [2]:
#Verilen bir hücrenin engel olup olmadığını kontrol eden fonk.
def engel_mi(gecerli_satir_indeks, gecerli_sutun_indeks):
  if oduller[gecerli_satir_indeks, gecerli_sutun_indeks] == -1.:
    return False
  else:
    return True

#Rastgele bir başlangıç konumu belirleyen fonk. Belirlenen konum bir engel olmamalıdır.
def baslangic_belirle():
  gecerli_satir_indeks = np.random.randint(ortam_satir_sayisi)
  gecerli_sutun_indeks = np.random.randint(ortam_sutun_sayisi)
  while engel_mi(gecerli_satir_indeks, gecerli_sutun_indeks):
    gecerli_satir_indeks = np.random.randint(ortam_satir_sayisi)
    gecerli_sutun_indeks = np.random.randint(ortam_sutun_sayisi)
  return gecerli_satir_indeks, gecerli_sutun_indeks

#epsilon-greedy politikası kullanarak bir sonraki hareketi belirleyen fonk. Satranç örn.
def sonraki_hareket_belirle(gecerli_satir_indeks, gecerli_sutun_indeks, epsilon):
  if np.random.random() < epsilon:
    return np.argmax(q_degerleri[gecerli_satir_indeks, gecerli_sutun_indeks])
  else: 
    return np.random.randint(4)

# **Robot future movement mechanism coding**

> **sonraki_noktaya_git** fonksiyonu sayesinde, robotun bir sonraki hareketi satır sütun ekseninde belirlenmekte; yeni hareket noktasına göre robotun yukarı hareketinde satır değeri azaltılıp, aşağı hareketinde artırılmakta, yine sağa harekette sütun değeri artırılıp, sola harekette azaltılmaktadır. 
> 
> **gecerli_satir_indeks ve gecerli_sutun_indeks:** Mevcut satır ve sütun indekslerini temsil eder.
> **hareket_indeks:** Gerçekleştirilecek hareketin indeksini belirtir *(0: yukarı, 1: sağ, 2: aşağı, 3: sol)*.
> **yeni_satir_indeks ve yeni_sutun_indeks:** Yeni konumun satır ve sütun indekslerini temsil eder.
> Hareketler kontrol edilir ve eğer hareket geçerli bir yönde yapılabiliyorsa (if koşulları), yeni indeksler güncellenir.Son olarak, yeni satır ve sütun indeksleri döner.

In [3]:
#Mevcut konumdan belirtilen bir hareketi gerçekleştirerek yeni bir konuma geçiş yapan fonk. 
#Hareketler "yukarı", "sağ", "aşağı" ve "sol" olarak tanımlanmıştır. 
def sonraki_noktaya_git(gecerli_satir_indeks, gecerli_sutun_indeks, hareket_indeks):
  yeni_satir_indeks = gecerli_satir_indeks
  yeni_sutun_indeks = gecerli_sutun_indeks
  if hareketler[hareket_indeks] == 'yukari' and gecerli_satir_indeks > 0:
    yeni_satir_indeks -= 1
  elif hareketler[hareket_indeks] == 'sag' and gecerli_sutun_indeks < ortam_sutun_sayisi - 1:
    yeni_sutun_indeks += 1
  elif hareketler[hareket_indeks] == 'asagi' and gecerli_satir_indeks < ortam_satir_sayisi - 1:
    yeni_satir_indeks += 1
  elif hareketler[hareket_indeks] == 'sol' and gecerli_sutun_indeks > 0:
    yeni_sutun_indeks -= 1
  return yeni_satir_indeks, yeni_sutun_indeks

# **Convenient path detection function**

> Robotun Q-Öğrenme ile eğitimi sonrasında belirlenmiş olan **en uygun** yolu verecek fonksiyon yazılır.
> 
> basla_satir_indeks ve basla_sutun_indeks: Başlangıç konumunun satır ve sütun indekslerini temsil eder. İlk olarak, başlangıç konumunun bir engel olup olmadığı kontrol edilir (*engel_mi* fonksiyonu kullanılarak). Eğer engelse, **boş bir liste** döner. Değilse, mevcut konum başlangıç konumu olarak ayarlanır ve **en_kisa listesi** bu konumu içerir.**Bir döngü içinde, mevcut konum engel veya hedef değilken:** *sonraki_hareket_belirle* fonksiyonu kullanılarak bir sonraki hareket belirlenir *(epsilon değeri 1 olarak verilir, yani tamamen keşif)*.
**sonraki_noktaya_git** fonksiyonu kullanılarak belirlenen hareket gerçekleştirilir ve yeni konum güncellenir.Yeni konum en_kisa listesine eklenir. Engel veya hedef bulunduğunda, **en_kisa listesi** döner.

In [4]:
#Belirtilen başlangıç konumundan bir engel veya hedefe ulaşana kadar en kısa yolu bulmayı amaçlar.
def en_kisa_mesafe(basla_satir_indeks, basla_sutun_indeks):
  if engel_mi(basla_satir_indeks, basla_sutun_indeks):
    return []
  else:
    gecerli_satir_indeks, gecerli_sutun_indeks = basla_satir_indeks, basla_sutun_indeks
    en_kisa = []
    en_kisa.append([gecerli_satir_indeks, gecerli_sutun_indeks])
    while not engel_mi(gecerli_satir_indeks, gecerli_sutun_indeks):
      hareket_indeks = sonraki_hareket_belirle(gecerli_satir_indeks, gecerli_sutun_indeks, 1.)
      gecerli_satir_indeks, gecerli_sutun_indeks = sonraki_noktaya_git(gecerli_satir_indeks, 
                                                                       gecerli_sutun_indeks, hareket_indeks)
      en_kisa.append([gecerli_satir_indeks, gecerli_sutun_indeks])
    return en_kisa

# **Q-Learning process codes**

> Son olarak, robotun **takviyeli öğrenmesini** sağlayacak; Q-Öğrenme ana fonksiyonu ve öğrenme parametreleri kodlanır.

> Q-Öğrenme döngüsü tanımlanmakta, buna göre **range** fonksiyonu içerisinde verilen değer **maksimum öğrenme adım sayısı** olmaktadır. 

> **engel_mi** fonksiyonundan faydalanılmaktadır. Yine her yeni harekette **sonraki_hareket_belirle** fonksiyonu çağırılarak robotun hareket ettirilmesi sağlanmaktadır. 

> **Q-Öğrenme formülü** üzerinden hesaplama yapılarak, başlangıçta boş tasarlanan **q_degerleri** adıyla temsil edilen *Q tablosunda* güncellemeler yapılmaktadır. 

> Eğitim sona erdiğinde ekrana *‘Eğitim tamamlandı.’* ibaresi yansıtılmaktadır. 

> **epsilon = 0.9:** Keşif (exploration) ve sömürü (exploitation) arasındaki dengeyi belirler. epsilon değeri yüksekse, ajan daha çok keşif yapar.
> **azalma_degeri = 0.9 (discount factor):** Gelecekteki ödüllerin bugünkü değerini belirler.
> **ogrenme_orani = 0.9 (learning rate):** Q-değerlerinin güncellenme hızını belirler.

> **Başlangıçta Yüksek Epsilon:** Eğitim sürecinin başında epsilon değeri genellikle yüksek tutulur. Bu, ajanın çevresini geniş ölçüde keşfetmesini sağlar.
> **Zamanla Azalan Epsilon:** Eğitim ilerledikçe epsilon değeri kademeli olarak azaltılır. Bu, ajanın giderek daha fazla sömürü yaparak öğrendiği bilgiyi kullanmasını sağlar.

> baslangic_belirle() fonksiyonu kullanılarak **rastgele** bir başlangıç noktası seçilir.
> Hedefe ulaşılana kadar döngü devam eder.
> **sonraki_hareket_belirle** fonksiyonu kullanılarak bir sonraki hareket belirlenir.
> **sonraki_noktaya_git** fonksiyonu ile ajan yeni konuma geçer.
> Yeni konumdaki ödül değeri alınır.
> Eski Q-değeri ve yeni Q-değerleri hesaplanır ve **Q-tablosu** güncellenir.

> Eğitim ile anlık ödüllerden ziyade, **gelecekteki** ödülleri de göz önünde bulundurarak karar vermeyi sağlar. Ajan, eğitim süresi boyunca **çevresini keşfeder** ve farklı durum-eylem çiftlerinden elde ettiği ödüllerle Q-tablosunu günceller. Bu sayede, ajan her duruma en uygun hareketi belirlemek için gerekli bilgiye sahip olur.

In [5]:
#Parametrelerin Tanımlanması
epsilon = 0.9
azalma_degeri = 0.9 
ogrenme_orani = 0.9 

#Eğitim Döngüsü 
#1000 adımdan oluşan bir döngüde Q-learning eğitimi yapılır.
for adim in range(1000):
  satir_indeks, sutun_indeks = baslangic_belirle()  
  while not engel_mi(satir_indeks, sutun_indeks):
    hareket_indeks = sonraki_hareket_belirle(satir_indeks, sutun_indeks, epsilon)
    eski_satir_indeks, eski_sutun_indeks = satir_indeks, sutun_indeks
    satir_indeks, sutun_indeks = sonraki_noktaya_git(satir_indeks, sutun_indeks, hareket_indeks)
    odul = oduller[satir_indeks, sutun_indeks] #Ödül ve Q-Değerlerinin Güncellenmesi
    eski_q_degeri = q_degerleri[eski_satir_indeks, eski_sutun_indeks, hareket_indeks]
    fark = odul + (azalma_degeri * np.max(q_degerleri[satir_indeks, sutun_indeks])) - eski_q_degeri
    yeni_q_degeri = eski_q_degeri + (ogrenme_orani * fark)
    q_degerleri[eski_satir_indeks, eski_sutun_indeks, hareket_indeks] = yeni_q_degeri
print('Eğitim tamamlandı.')

Eğitim tamamlandı.


# **Appropriate road/route determination based on starting point**

> Eğitilen robot Q tablosunda kendisini yeşil kareye götürecek uygun rotaları belirlemiş olduğu için kullanıcıdan herhangi bir başlangıç noktası (satır ve sütun indeks değerine göre) istenerek kargonun indirilmesi için izlenecek yolun/rotanın ekrana yazdırılması sağlanır.

> Kullanıcıdan başlangıç satır indeksi ve başlangıç sütun indeksi istenmektedir. Kullanıcıdan alınan bilgiler ışığında, **en_kisa_mesafe** fonksiyonu kullanılmak suretiyle robotun yeşil karede kargo indirmek için izleyeceği rota ekrana yazdırılmaktadır.

In [6]:
#Eğitim Sonrası Test
#Kullanıcıdan Başlangıç Noktası Alınması
ogr_sonrasi_satir = input('Robotun harekete başlayacağı satır indeksini giriniz:')
ogr_sonrasi_sutun = input('Robotun harekete başlayacağı sütun indeksini giriniz:')

#En Kısa Mesafenin Hesaplanması ve Yazdırılması
print('Kargo noktasına giden rota:',
      en_kisa_mesafe(int(ogr_sonrasi_satir), int(ogr_sonrasi_sutun)))

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.